# 지하철역과 법정동 연결

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 1. 데이터 읽기

In [211]:
subway_df = pd.read_csv('../4.Public_transportation/station_address.csv', encoding="CP949")
l_dong_df = pd.read_csv('forDP/LdongID.csv', encoding="CP949")
seoul_subway_df = pd.read_csv('../4.Public_transportation/서울시 지하철 역별 승하차 인원.csv',
                              encoding="CP949")

In [212]:
subway_df.info(), l_dong_df.info(), seoul_subway_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    289 non-null    object
 1   name    289 non-null    object
 2   city    289 non-null    object
 3   gu      289 non-null    object
 4   dong    289 non-null    object
dtypes: object(5)
memory usage: 11.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      467 non-null    int64 
 1   gu      467 non-null    object
 2   dong    467 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24698 entries, 0 to 24697
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사용일자    24698 non-null  int64 
 1   호선명     24698 non-null  object
 2   역명      246

(None, None, None)

In [213]:
seoul_subway_df.columns = ['date', 'line', 'name', 'boarding', 'alighting', 'date_data']
seoul_subway_df = seoul_subway_df[['line', 'name']]

In [214]:
seoul_subway_df = seoul_subway_df.drop_duplicates()
seoul_subway_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 620 entries, 0 to 5287
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    620 non-null    object
 1   name    620 non-null    object
dtypes: object(2)
memory usage: 14.5+ KB


In [215]:
seoul_subway_df.groupby('line')['name'].count().sort_index()

line
1호선         10
2호선         50
3호선         34
4호선         26
5호선         56
6호선         39
7호선         42
8호선         19
9호선         25
9호선2~3단계    13
경강선         12
경부선         39
경원선         33
경의선         29
경인선         20
경춘선         19
공항철도 1호선    14
과천선          8
분당선         35
서해선          3
수인선         18
신림선         11
안산선         13
우이신설선       13
일산선         11
장항선          7
중앙선         21
Name: name, dtype: int64

In [216]:
seoul_subway_df['line'] = seoul_subway_df['line'].map(lambda v : str(v).replace('9호선2~3단계', "9호선"))

In [217]:
seoul_subway_df.groupby('line')['name'].count().sort_index()

line
1호선         10
2호선         50
3호선         34
4호선         26
5호선         56
6호선         39
7호선         42
8호선         19
9호선         38
경강선         12
경부선         39
경원선         33
경의선         29
경인선         20
경춘선         19
공항철도 1호선    14
과천선          8
분당선         35
서해선          3
수인선         18
신림선         11
안산선         13
우이신설선       13
일산선         11
장항선          7
중앙선         21
Name: name, dtype: int64

In [218]:
subway_df.groupby('line')['name'].count().sort_index()

line
1호선    10
2호선    50
3호선    34
4호선    26
5호선    56
6호선    39
7호선    42
8호선    19
9호선    13
Name: name, dtype: int64

In [220]:
# 각 데이터프레임의 line별 역 목록 생성
def get_stations_by_line(df):
    return {line: set(group['name']) for line, group in df.groupby('line')}

# 두 데이터프레임의 line별 역 목록 생성
seoul_stations = get_stations_by_line(seoul_subway_df)
all_stations = get_stations_by_line(subway_df)

# 공통된 line들 찾기
common_lines = set(seoul_stations.keys()) & set(all_stations.keys())

# line별로 비교
for line in sorted(common_lines):
    seoul_set = seoul_stations[line]
    all_set = all_stations[line]
    
    print(f"\n{line} 비교:")
    print(f"역 개수: seoul_subway_df {len(seoul_set)}개, subway_df {len(all_set)}개")
    
    # 차이점 있는 경우만 출력
    only_in_seoul = seoul_set - all_set
    only_in_all = all_set - seoul_set
    
    if only_in_seoul:
        print(f"seoul_subway_df에만 있는 역:", sorted(only_in_seoul))
    if only_in_all:
        print(f"subway_df에만 있는 역:", sorted(only_in_all))
    if not only_in_seoul and not only_in_all:
        print("두 데이터프레임의 역이 완전히 일치합니다.")


1호선 비교:
역 개수: seoul_subway_df 10개, subway_df 10개
seoul_subway_df에만 있는 역: ['서울역']
subway_df에만 있는 역: ['서울']

2호선 비교:
역 개수: seoul_subway_df 50개, subway_df 50개
seoul_subway_df에만 있는 역: ['교대(법원.검찰청)', '신촌']
subway_df에만 있는 역: ['교대(법원, 검찰청)', '신촌(지하)']

3호선 비교:
역 개수: seoul_subway_df 34개, subway_df 34개
seoul_subway_df에만 있는 역: ['교대(법원.검찰청)']
subway_df에만 있는 역: ['교대(법원, 검찰청)']

4호선 비교:
역 개수: seoul_subway_df 26개, subway_df 26개
seoul_subway_df에만 있는 역: ['미아사거리', '서울역']
subway_df에만 있는 역: ['미아사거리 ', '서울']

5호선 비교:
역 개수: seoul_subway_df 56개, subway_df 56개
seoul_subway_df에만 있는 역: ['종로3가']
subway_df에만 있는 역: ['종로3가(탑골공원)']

6호선 비교:
역 개수: seoul_subway_df 39개, subway_df 39개
두 데이터프레임의 역이 완전히 일치합니다.

7호선 비교:
역 개수: seoul_subway_df 42개, subway_df 42개
seoul_subway_df에만 있는 역: ['자양(뚝섬한강공원)']
subway_df에만 있는 역: ['자양(뚝섬유원지)']

8호선 비교:
역 개수: seoul_subway_df 19개, subway_df 19개
seoul_subway_df에만 있는 역: ['남한산성입구(성남법원.검찰청)', '암사역사공원']
subway_df에만 있는 역: ['남한산성입구(성남법원?검찰청)', '암사역사공원역']

9호선 비교:
역 개수: seoul_subway_df 38개, sub

In [221]:
def clean_station_name(name):
    # 공백 제거
    name = name.strip()
    # '역' 제거 (끝에 있는 경우)
    name = name.rstrip('역')
    # 괄호 안 구분자 통일
    name = name.replace('?', '.').replace(', ', '.')
    # 기타 정규화 작업
    name = name.replace('서울역', '서울')
    name = name.replace('서울', '서울역')
    name = name.replace('신촌(지하)', '신촌')
    name = name.replace('신촌', '신촌(지하)')
    name = name.replace('종로3가(탑골공원)', '종로3가')
    name = name.replace('종로3가', '종로3가(탑골공원)')
    name = name.replace('자양(뚝섬유원지)', '자양(뚝섬한강공원)')

    return name

# 데이터프레임에 적용
seoul_subway_df['name'] = seoul_subway_df['name'].apply(clean_station_name)
subway_df['name'] = subway_df['name'].apply(clean_station_name)

In [222]:
# 각 데이터프레임의 line별 역 목록 생성
def get_stations_by_line(df):
    return {line: set(group['name']) for line, group in df.groupby('line')}

# 두 데이터프레임의 line별 역 목록 생성
seoul_stations = get_stations_by_line(seoul_subway_df)
all_stations = get_stations_by_line(subway_df)

# 공통된 line들 찾기
common_lines = set(seoul_stations.keys()) & set(all_stations.keys())

# line별로 비교
for line in sorted(common_lines):
    seoul_set = seoul_stations[line]
    all_set = all_stations[line]
    
    print(f"\n{line} 비교:")
    print(f"역 개수: seoul_subway_df {len(seoul_set)}개, subway_df {len(all_set)}개")
    
    # 차이점 있는 경우만 출력
    only_in_seoul = seoul_set - all_set
    only_in_all = all_set - seoul_set
    
    if only_in_seoul:
        print(f"seoul_subway_df에만 있는 역:", sorted(only_in_seoul))
    if only_in_all:
        print(f"subway_df에만 있는 역:", sorted(only_in_all))
    if not only_in_seoul and not only_in_all:
        print("두 데이터프레임의 역이 완전히 일치합니다.")


1호선 비교:
역 개수: seoul_subway_df 10개, subway_df 10개
두 데이터프레임의 역이 완전히 일치합니다.

2호선 비교:
역 개수: seoul_subway_df 50개, subway_df 50개
두 데이터프레임의 역이 완전히 일치합니다.

3호선 비교:
역 개수: seoul_subway_df 34개, subway_df 34개
두 데이터프레임의 역이 완전히 일치합니다.

4호선 비교:
역 개수: seoul_subway_df 26개, subway_df 26개
두 데이터프레임의 역이 완전히 일치합니다.

5호선 비교:
역 개수: seoul_subway_df 56개, subway_df 56개
두 데이터프레임의 역이 완전히 일치합니다.

6호선 비교:
역 개수: seoul_subway_df 39개, subway_df 39개
두 데이터프레임의 역이 완전히 일치합니다.

7호선 비교:
역 개수: seoul_subway_df 42개, subway_df 42개
두 데이터프레임의 역이 완전히 일치합니다.

8호선 비교:
역 개수: seoul_subway_df 19개, subway_df 19개
두 데이터프레임의 역이 완전히 일치합니다.

9호선 비교:
역 개수: seoul_subway_df 38개, subway_df 13개
seoul_subway_df에만 있는 역: ['가양', '개화', '고속터미널', '공항시장', '구반포', '국회의사당', '김포공항', '노들', '노량진', '당산', '동작(현충원)', '등촌', '마곡나루(서울역식물원)', '사평', '샛강', '선유도', '신논현', '신목동', '신반포', '신방화', '양천향교', '여의도', '염창', '증미', '흑석(중앙대입구)']


seoul_subway_df만 사용 결정

In [223]:
seoul_subway_df.groupby('line')['name'].count().sort_index()

line
1호선         10
2호선         50
3호선         34
4호선         26
5호선         56
6호선         39
7호선         42
8호선         19
9호선         38
경강선         12
경부선         39
경원선         33
경의선         29
경인선         20
경춘선         19
공항철도 1호선    14
과천선          8
분당선         35
서해선          3
수인선         18
신림선         11
안산선         13
우이신설선       13
일산선         11
장항선          7
중앙선         21
Name: name, dtype: int64

In [138]:
non_seoul = subway_df[subway_df['city'] != "서울특별시"]
non_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 60 to 275
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    15 non-null     object
 1   name    15 non-null     object
 2   city    15 non-null     object
 3   gu      15 non-null     object
 4   dong    15 non-null     object
dtypes: object(5)
memory usage: 720.0+ bytes


In [139]:
seoul_df = subway_df[subway_df['city'] == "서울특별시"]
seoul_df.info(), seoul_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, 0 to 288
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    274 non-null    object
 1   name    274 non-null    object
 2   city    274 non-null    object
 3   gu      274 non-null    object
 4   dong    274 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


(None,
   line  name   city   gu                  dong
 0  1호선    서울  서울특별시   중구  남대문로5가 73-6 서울역(1호선)
 1  1호선    시청  서울특별시   중구       정동 5-5 시청역(1호선)
 2  1호선    종각  서울특별시  종로구      종로1가 54 종각역(1호선)
 3  1호선  종로3가  서울특별시  종로구  종로3가 10-5 종로3가역(1호선)
 4  1호선  종로5가  서울특별시  종로구  종로5가 82-1 종로5가역(1호선))

In [140]:
# 1. seoul_df의 dong 컬럼 전처리 (주소에서 동 이름만 추출)
def extract_dong(address):
    # 공백으로 분리하고 첫 번째 단어(동 이름이 있는 부분)만 가져옴
    dong_part = address.split()[0]
    # 하이픈 이후의 문자열 제거
    dong_name = dong_part.split('-')[0]
    return dong_name

In [141]:
# dong 컬럼 전처리
seoul_df['dong_clean'] = seoul_df['dong'].apply(extract_dong)

C:\Users\human\AppData\Local\Temp\ipykernel_4604\1697857905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_df['dong_clean'] = seoul_df['dong'].apply(extract_dong)


In [142]:
# 2. 법정동 정보 추가
seoul_df['legal_dong'] = None

C:\Users\human\AppData\Local\Temp\ipykernel_4604\1446007410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_df['legal_dong'] = None


In [143]:
# 3. 각 행에 대해 매칭되는 법정동 찾기
for idx, row in seoul_df.iterrows():
    matching_dong = l_dong_df[(l_dong_df['gu'] == row['gu']) & 
                             (l_dong_df['dong'] == row['dong_clean'])]
    if not matching_dong.empty:
        seoul_df.at[idx, 'legal_dong'] = matching_dong.iloc[0]['dong']

In [144]:
# 4. 임시로 만든 dong_clean 컬럼 제거
seoul_df = seoul_df.drop('dong_clean', axis=1)

# 5. dong 컬럼 제거
seoul_df = seoul_df.drop('dong', axis=1)

In [145]:
seoul_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, 0 to 288
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   line        274 non-null    object
 1   name        274 non-null    object
 2   city        274 non-null    object
 3   gu          274 non-null    object
 4   legal_dong  274 non-null    object
dtypes: object(5)
memory usage: 20.9+ KB


In [146]:
# 결과 확인
print(seoul_df.head(10))

  line          name   city    gu legal_dong
0  1호선            서울  서울특별시    중구     남대문로5가
1  1호선            시청  서울특별시    중구         정동
2  1호선            종각  서울특별시   종로구       종로1가
3  1호선          종로3가  서울특별시   종로구       종로3가
4  1호선          종로5가  서울특별시   종로구       종로5가
5  1호선           동대문  서울특별시   종로구        창신동
6  1호선           신설동  서울특별시  동대문구        신설동
7  1호선           제기동  서울특별시  동대문구        제기동
8  1호선  청량리(서울시립대입구)  서울특별시  동대문구        전농동
9  1호선           동묘앞  서울특별시   종로구        숭인동


In [147]:
# legal_dong별 지하철 역 개수 계산
station_count = seoul_df.groupby('legal_dong')['name'].count().sort_values(ascending=False)

# 결과 확인
print("법정동별 지하철 역 개수:")
print(station_count)

법정동별 지하철 역 개수:
legal_dong
상계동      6
삼성동      6
신당동      6
가락동      5
서초동      5
        ..
보문동1가    1
문정동      1
문래동3가    1
묵동       1
흥인동      1
Name: name, Length: 164, dtype: int64


In [148]:
# gu별 지하철 역 개수 계산
station_count_by_gu = seoul_df.groupby('gu')['name'].count().sort_values(ascending=False)

# 결과 확인
print("구별 지하철 역 개수:")
print(station_count_by_gu)

구별 지하철 역 개수:
gu
송파구     28
중구      23
강남구     21
마포구     16
종로구     15
강동구     15
성동구     14
은평구     13
노원구     13
서초구     12
동작구     12
영등포구    11
광진구     11
용산구     10
성북구      9
강서구      9
중랑구      8
구로구      7
동대문구     6
서대문구     5
양천구      5
관악구      4
도봉구      3
강북구      3
금천구      1
Name: name, dtype: int64
